# Persephone

**PLEASE READ THIS NOTEBOOK IN ITS ENTIRETY BEFORE RUNNING**

This notebook facilitates the creation of a *gdastool* type of atmospheric profile for being used in CORSIKA simulations. Instead of just runninig *gdastool* to obtain a single atmospheric profile for a specific time and date, this code creates an **average** monthly profile.

It consists of **2 blocks** as follows:

- **Block 1** uses *gdastool* to create atmospheric profiles throughout the month chosen by the user, two profiles per day.
- **Block 2** uses all the profiles created on block 1 and creates an average atmospheric profile for the given month. This profile is a **.dat** file that can be input in a CORSIKA simulation.

## Requisites

For this code to work, the user must download the directory called **scripts**, where the scripts used to create the atmospheric profile are found. These are:

- **gdastool**, a *Python* program normally included with newer veresions of CORSIKA, specifically designed to create CORSIKA atmospheric profiles based on data from the **Global Data Assimilation System*.
- **first_half.C** and **second_half.C** are *root* based scripts that will create one single monthly-average atmospheric profile using all the individual profiles preveiously created throught the chosen month.

The computer where this notebook is to be run must have **CERN's ROOT** built in it, since the *.C* scripts need it to work. Check out the official web page to learn how to download and build ROOT: https://root.cern.ch/building-root. The environment must be correctly setup before running this notebook. To do this, follow the instructions given on the web page, on the *Quick Start* section.

In [ ]:
# These are the libraries that will be needed to run the notebook

import subprocess
import numpy as np
import time
from datetime import datetime
from datetime import timezone

In [ ]:
# First, the user must input the path to the scripts directory

print("Please, write down the path to the scripts directory from the home folder. It must end in '/scripts' without a '/' at the end.")
path2Scripts = input("The path to the scripts directory from home folder is: ~/")

## Block 1 - GDASTOOL

CORSIKA uses atmospheric density profiles to calculate the probability of interactions throughout the showers. Density is modeled by dividing the atmosphere in 5 layeres, describing each one of them with the following equations:

- For the first 4 layers: $$T(h)=a_i+b_ie^{\frac{-h}{c_i}} \quad i=1,2,3,4$$
- For the fifth layer: $$T(h)=a_5-b_5\frac{h}{c_5}$$

Here, *T(h)* is the atmospheric density at height *h*, and *a*, *b* and *c* are parameters specific to each layer.

In this notebook, we use the Python program *gdastool* to generate an output file called *Atmosphere.dat*, readable by CORSIKA, containing all the information about the layer division, the parameters *a*, *b* and *c*, as well as a tabulated altitude profile of the atmospheric refractive index.

*gdastool* mainly takes as input the geographic coordinates of the place where the simulation is to take place (latitude and longitude), and a UTC timestamp. The coordinates are rounded to the nearest 1x1 degree grid point, and the timestamp is rounded to the nearest 3-hour time in GDAS.

To use *gdastool*, this next section will ask the user to write the inputs needed: geographic coordinates (in decimal degrees) and a date and UTC time, which will later be converted to a UTC timestamp.

In [ ]:
# First, we ask the user to introduce the geographic coordinates
print("""Please write the geographic coordinates of the place for simulation in decimal degrees.
For example, coordinates for the White House would be written like this:
Latitude: 38.8978
Longitude: -77.0364""")
lat = input("Latitude (dd.dddd): ")
long = input("Longitude (dd.dddd): ")

# Then, the user must write the date and time
print("Now please provide the date (month and year) of simulation for the creation of the atmospheric profile.")
month, year = input("Date (mm/yyyy): ").split("/")

# Then, the user must write the timezone used on the place where the simulation is to take place.
print("Now please provide the timezone used on the location at the time of simulation. For example, Colombia uses UTC-5")
simuTimezone = input("Timezone: UTC")

# Date values must be converted from str to integers
year, month = int(year), int(month)

# The timezone value must be converted to an integer
simuTimezone = int(simuTimezone)

# Latitude and longitude values must be converted from str to float
lat, long = float(lat), float(long)

Instead of just using a GDAS atmospheric profile for a specific time, this code creates an average atmospheric profile for the given month. The way it works is as follows:
- First, it determines the timestamps for the frist day of the given month in the given year at midnight on the given timezone.
- Then, it creates a series of atmospheric profiles, every 12 hours, starting from the aforementioned timestamp and ending on the last day of the given month, at noon on the given timezone.

In other words, this code creates an atmospheric profile on the given location twice a day (at midnight and noon) throughout the whole month given by the user, for a total of 62 atmospheric profiles (if the month has 31 days) or 60, 58 or 56 (if the month has 30, 29 or 28 days, respectively).

In [ ]:
# These lines determine the number of the days the given month has, and create the starting and finishing dates.

firstDay = 1

if month in (1,3,5,7,8,10,12):
    lastDay = 31

elif month in (4,6,9,11):
    lastDay = 30

elif month == 2:
    if year%4 != 0 or (year%100 == 0 and year%400 != 0):
        lastDay = 28
    elif year%400 == 0 or (year%4 == 0 and year%100 != 0):
        lastDay = 29

startTime = datetime(year,month,firstDay,0,0)
finishTime = datetime(year,month,lastDay,12,0)

Since *gdastool* takes a UTC timestamp as input, the date and time given by the user must be converted to that.

In [ ]:
# This part takes the date and time given by the user and converts them to a UTC timestamp

utc_time1 = startTime.replace(tzinfo = timezone.utc)
utc_timestamp1 = int(utc_time1.timestamp())-3600*simuTimezone

utc_time2 = finishTime.replace(tzinfo = timezone.utc)
utc_timestamp2 = int(utc_time2.timestamp())-3600*simuTimezone

print(utc_timestamp1,utc_timestamp2)

To create the series of atmospheric profiles, this next cell creates a file called *GDASrunner.sh* containing a bash script that will run GDAS with the specified inputs given by the user.

This next cell will also create a few more bash scripts for managing the files (copying, pasting and moving the files automatically).

In [ ]:
# This part creates the file 'GDASrunner.sh' that will create the series of atmospheric profiles
GDASrunner = open("GDASrunner.sh", "w")
GDASrunner.write('#!/bin/bash\n')
GDASrunner.write('echo Corriendo gdastool...\n')
GDASrunner.write('for i in {'+str(utc_timestamp1)+'..'+str(utc_timestamp2)+'..43200}; do python3 ~/'+path2Scripts+'/gdastool -t $i -o Atm_$i.dat -c '+str(lat)+' '+str(long)+' -p gdas_path\n')
GDASrunner.write('\techo Creando perfiles para la UNIXTIME: $i\n')
GDASrunner.write('\tsleep 1m\n')
GDASrunner.write('\techo Perfil para $i terminado...\n')
GDASrunner.write('done')
GDASrunner.close()

#This part creates a bash script that copies the scripts that create the average profile into the current folder
copy2Folder = open("copy2Folder.sh", "w")
copy2Folder.write('#!/bin/bash\n')
copy2Folder.write('cp ~/'+path2Scripts+'/primera_parte.C .\n')
copy2Folder.write('cp ~/'+path2Scripts+'/segunda_parte.C .\n')
copy2Folder.close()

#This part creates a script that combines the two average files into one.
paste = open("paste.sh", "w")
paste.write('#!/bin/bash\n')
paste.write('cat primera_parte.txt segunda_parte.txt > Atm_'+str(month)+'_'+str(year)+'.dat\n')
paste.close()

#This part creates a script that moves the atmospheric profiles into one folder and gets rid of all the
#files that aren't necessary anymore.
copy2Folder2 = open("copy2Folder2.sh", "w")
copy2Folder2.write('#!/bin/bash\n')
copy2Folder2.write('mkdir '+str(month)+'_'+str(year)+'\n')
copy2Folder2.write('rm primera_parte.C\n')
copy2Folder2.write('rm segunda_parte.C\n')
copy2Folder2.write('rm primera_parte.txt\n')
copy2Folder2.write('rm segunda_parte.txt\n')
copy2Folder2.write('rm copy2Folder.sh\n')
copy2Folder2.write('rm paste.sh\n')
copy2Folder2.write('rm GDASrunner.sh\n')
copy2Folder2.write('for i in {'+str(utc_timestamp1)+'..'+str(utc_timestamp2)+'..43200}; do mv Atm_$i.dat ./'+str(month)+'_'+str(year)+'/\n')
copy2Folder2.write('done')
copy2Folder2.close()

With the required inputs given by the user, this next section executes *GDASrunner.sh*.

In [ ]:
# This part runs the GDASrunner.sh script that runs gdastool multiple times to create the series of atm. profiles
runGDAS = subprocess.run(["bash", "GDASrunner.sh"])

## Block 2 - Average profile

Once all the files have been moved, this block will use them to create a single average atmospheric profile. This part uses the C++ scripts called *first_half.C* and *second_half.C*. The validity of CORSIKA simulations using this monthly-average atmspheric profile has been proven as shown in (Grisales, J. 2019. *Universidad Industrial de Santander*. Undergrad. Thesis).

For this block to work, please make sure the ROOT environment is setup beforehand, i. e. the command *root* should be recognised by the same Terminal used to run this notebook.

In [ ]:
# This part will use the scripts created above to create one single average atmospheric profile

runCopy = subprocess.run(["bash", "copy2Folder.sh"])
runFirstHalf = subprocess.run(["root", "primera_parte.C"])
runsecondHalf = subprocess.run(["root", "segunda_parte.C"])
paste = subprocess.run(["bash", "paste.sh"])

#This final part does some cleaning. It gets rid of unnecessary files and moves all the individual atmospheric
#profiles into one folder
move = subprocess.run (["bash", "copy2Folder2.sh"])
rm = subprocess.run(["rm", "copy2Folder2.sh"])

The resulting file is called **Atm_mm_yyyy.dat**, where mm and yyyy are the month and year chosen by the user, respectively. That file is the average of all the other atmospheric profiles created throughout the whole month, two per day. **Atm_mm_yyyy.dat** can be directly used as input for CORSIKA-76400 simulations.